In [10]:
__author__ = "Zhenming Xu"
__email__ = "15216105346@163.com"
__date__ = "Aug 8, 2019"

In [11]:
from pandas import DataFrame
from functools import reduce
import numpy as np
from pymatgen.ext.matproj import MPRester
from pymatgen.core.periodic_table import Element
%matplotlib inline

In [ ]:
mp_api = MPRester()
binary_entries = mp_api.query('Li-S', properties = ['elements', 'nelements', 'pretty_formula', 'spacegroup.symbol', 'material_id', 'e_above_hull'])

pmg_all_els = set([Element.from_Z(i).symbol for i in range(1, 83)])
elements = list(pmg_all_els)
remove_elements = ['He','Ne','Ar','Kr','Xe','Rn']#['O','S','Se','Te','F','Cl','Br','I','He','Ne','Ar','Kr','Xe','Rn']
for i in elements:
    for j in remove_elements:
        if j in elements:
            elements.remove(j)

ternary_entries = []
for B_element in elements:
    ternary_entries = ternary_entries + mp_api.query('Li-S-{}'.format(B_element), properties = ['elements', 'nelements', 'pretty_formula', 'spacegroup.symbol', 'material_id', 'e_above_hull'])    

quaternary_entries = []
for B_element in elements:
    for C_element in elements:
        quaternary_entries = quaternary_entries + mp_api.query('Li-S-{}-{}'.format(B_element, C_element), properties = ['elements', 'nelements', 'pretty_formula', 'spacegroup.symbol', 'material_id', 'e_above_hull'])

In [ ]:
total_entries = binary_entries + ternary_entries + quaternary_entries
def list_dict_duplicate_removal():
    run_function = lambda x, y: x if y in x else x + [y]
    return reduce(run_function, [[], ] + total_entries)
total = DataFrame(list_dict_duplicate_removal())
total_stable = total.loc[(total['e_above_hull'] <= 0.05) & (total['nelements'] >= 2)].sort_values('e_above_hull').reset_index(drop=True)
total_stable.to_csv('stable_lithium_sulfides_from_MP_database.csv')